In [1]:
# Arthur Ryan
# NCI-AIML course
# Student Number #20170386

# Bayesian Regression python code

from scipy import stats
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn.linear_model import Ridge
from array import *
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn as sns

from google.colab import drive

import os
path = os.path.abspath(os.curdir)

# for Development / Student use
# from google.colab import drive
# drive.mount('/content/drive/')
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project-Regression/Project-AI-Mockaroo_training_data_v1.1_BayesRegression.csv') # to match BN code line


# *** for Examiner / for submission
df = pd.read_csv('Project-AI-Mockaroo_training_data_v1.1_BayesRegression.csv') # i.e. for same directory as python code i.e. the data file is in the same directory as the python code
# df = pd.read_csv(path, 'Project-AI-Mockaroo_training_data_v1.1_BayesRegression.csv') # i.e. for same directory as python code i.e. the data file is in the same directory as the python code
# df = pd.read_csv('data.csv') # i.e. for same directory as python code i.e. the data file is in the same directory as the python code


# import csv
# with open('Project-AI-Mockaroo_training_data_v1.1_BayesRegression.csv', 'r') as file: # i.e. for same directory as python code i.e. the data file is in the same directory as the python code
#    reader = csv.reader(file)
#    df = pd.DataFrame(reader)



for i in range (0, 25000):
    # X = df[['TotalMissionCost','Unused_Capacity','Cycle time (days)','Ship hire cost per day','Arrival time']] # .loc[0:25000] # does not work ->  .iloc[:,-1]
    X = df[['CostCycleTimeShipHire','Unused_Capacity','Cycle time (days)','Ship hire cost per day','Arrival time']] # .loc[0:25000] # does not work ->  .iloc[:,-1]

# *** Correlation Matrix and graph
X_corr = round(X.corr(),2)
print('\nCorrelation Matrix\n',X_corr)

# X_corr = X.corr()
axis_corr = sns.heatmap(
X_corr,
vmin=-1, vmax=1, center=0,
cmap=sns.diverging_palette(50, 500, n=500),
square=True 
)

data_array = np.asarray(X)

#print('\n*** X = ')
#print(X)
#print('\n*** data_array = ')
#print(data_array)

# Generating simulated data with Gaussian weights
# np.random.seed(0)

n_samples, n_features = 5, 5 # n_features changed to 1 as count starts at 0 below

# Create weights with a precision lambda_ of 4.
lambda_ = 4.
w_temp = np.zeros((n_features,n_samples))
w = np.zeros((n_features,n_samples))
#print('\nw = np.zeros((n_features, n_samples))')
#print(w_temp)

# Only keep 5 weights of interest
# relevant_features = np.random.randint(0, n_features, 1)
relevant_features = [0,1,2,3,4]
for j in relevant_features:
  for i in range(0,5,1):
      w_temp[j][i] = stats.norm.rvs(loc=0, scale=1. / np.sqrt(lambda_))
      w[j][i] = abs(w_temp[j][i])

#print('\nw_temp[j][i] = ',w_temp)
#print('\nw[j][i] = ',w)

# Create noise with a precision alpha of 50.
alpha_ = 100. # was 50 in previous iteration
noise = stats.norm.rvs(loc=0, scale=1. / np.sqrt(alpha_), size=n_samples)

# print('\nnoise = ',noise)

y = np.dot(data_array, w) + noise

#print('\ny = np.dot(data_array, w)\n = ',y)

# #############################################################################
# Fit the Bayesian Ridge Regression and an OLS for comparison
clf = BayesianRidge(fit_intercept=True, compute_score=True)
# clf.fit(X, y)

#print ('\nX.shape[0] = ', X.shape[0])
#print ('Y.shape[0] = ', y.shape[0])

#print ('\nX.shape[1] = ', X.shape[1])
#print ('Y.shape[1] = ', y.shape[1],'\n')

# *** flattening the y output 2 dimension array to a one dimaension vector for bayesian regression fitting
y_temp = (y[:,0] + y[:,1] + y[:,2] + y[:,3] + y[:,4])  # trying to make a scalar of the 25,000 rows times 5 columns of data

# y_temp.shape[0]

# print('\ny_temp',y_temp,'\n')

clf.set_params(n_iter=10,)

clf.fit(data_array, y_temp)

ols = LinearRegression()

ols.fit(data_array, y_temp)

# Parameters list
# print('\nParameters list = ',clf. get_params(deep=True))
# print('\nParameter estimators = ',clf.set_params(**params))¶

# #############################################################################
# Visualisations
# Plot true weights, estimated weights, histogram of the weights, and
# predictions with standard deviations

# Value of the weights - graph
lw = 2
plt.figure(figsize=(6, 6))
plt.title("Weights of the model")
plt.plot(clf.coef_, color='lightgreen', linewidth=lw,
         label="Bayesian Ridge estimate")
plt.plot(w, color='gold', linewidth=lw, label="Ground truth")
plt.plot(ols.coef_, color='navy', linestyle='--', label="OLS estimate")
plt.xlabel("Features")
plt.ylabel("Values of the weights")
plt.legend(loc="best", prop=dict(size=12))


# Histogram of the weights - graph
plt.figure(figsize=(6, 6))
plt.title("Histogram of the weights")
plt.hist(clf.coef_, bins=n_features, color='gold', log=True,
         edgecolor='black')
plt.scatter(clf.coef_[relevant_features], np.full(len(relevant_features), 5.),
            color='navy', label="Relevant features")
plt.ylabel("Features")
plt.xlabel("Values of the weights")
plt.legend(loc="upper left")

# Marginal log likelihood - graph
plt.figure(figsize=(6, 6))
plt.title("Marginal log-likelihood")
plt.plot(clf.scores_, color='navy', linewidth=lw)
plt.ylabel("Score")
plt.xlabel("Iterations")


# Plotting some predictions for polynomial regression
# Plotting some predictions for polynomial regression
# Plotting some predictions for polynomial regression
def f(x, noise_amount):
    y = np.sqrt(x) * np.sin(x)
    noise = np.random.normal(0, 1, len(x))
    return y + noise_amount * noise

degree = 10
X = np.linspace(0, 10, 100)
y = f(X, noise_amount=0.1)

clf_poly = BayesianRidge()
clf_poly.fit(np.vander(X, degree), y)

X_plot = np.linspace(0, 11, 25)
y_plot = f(X_plot, noise_amount=0)
y_mean, y_std = clf_poly.predict(np.vander(X_plot, degree), return_std=True)
plt.figure(figsize=(6, 6))
plt.errorbar(X_plot, y_mean, y_std, color='navy',
             label="Polynomial Bayesian Ridge Regression", linewidth=lw)
plt.plot(X_plot, y_plot, color='gold', linewidth=lw,
         label="Ground Truth")
plt.ylabel("Output - Value of Weights")
plt.xlabel("Feature 1 to 5: CostCycleTime(SH) ; CycleTime ; ShipHire ; TimeNow ; TimeArrival")
plt.legend(loc="lower left")
plt.show()

# Predict using the Bayesian regression model.
clf.score(data_array, y_temp) # -> Return the coefficient of determination  of the prediction
print('\n*** CLF score = ',clf.score(data_array, y_temp))

# *** Prediction ***
X = [[100000, 10000, 5, 10000, 18],
    [150000, 15000, 10, 20000, 22] ]
print('\nPrediction = ',clf.predict(X,return_std=True))


# *** trimming decimal places ***  formatting example
# *** trimming decimal places ***  formatting example
# *** trimming decimal places ***  formatting example

# full precision no formatting added
#print(y_temp)
#print("\n'Bayesian Model Output/Dependent Variable statistics:'")
#print('\nDescribe summary statistics listing = \n',stats.describe(y_temp))
#print('\nStandard Deviation = ',stats.tstd(y_temp))
#print('Standard Error of the Mean = ',stats.tsem(y_temp))
#print('Coefficient of Variation (stdev / mean) = ',stats.variation(y_temp))
#print('Trimmed by 10% Mean = ',stats.trim_mean(y_temp,0.10))
#print('Inter Quartile Range = ',stats.iqr(y_temp))

#TMC_array = np.array(df['TotalMissionCost'])
#print('ANOVA results = ',stats.f_oneway(y_temp, TMC_array))
#print('Correlation results = ',stats.pearsonr(y_temp, TMC_array))
#Sum_of_errors = (y_temp - TMC_array)
#print('Average Error in Euros = ', np.mean(Sum_of_errors))
#print("Average Error in % of TotalMissionCost mean = ", np.mean(abs(Sum_of_errors)) / np.mean(TMC_array),'%')
#print("Average Error in % of Model Output mean = ", np.mean(abs(Sum_of_errors)) / np.mean(y_temp),'%')
#print("Standard Error in Euros = ", stats.tsem(Sum_of_errors))
#print("Standard Error in % of Avg Error= ", stats.tsem(Sum_of_errors) / np.mean(Sum_of_errors),'%')


print("\n'Bayesian Model Output/Dependent Variable statistics:'")
print('\nDescribe summary statistics listing = \n',stats.describe(y_temp))

# print('*** VERSION TWO OUTPUT ***')
print("\n**** 'Model Output y Variable' statistics:' ***")

sdev = stats.tstd(y_temp)
txt1 = '\nStandard Deviation = €{:,.2f}'
print(txt1.format(sdev))

serror = stats.tsem(y_temp)
txt2 = '\nStandard Error of the Mean = €{:,.2f}'
print(txt2.format(serror))

coeffvartnmean = stats.variation(y_temp)
txt3 = '\nCoefficient of Variation (stdev / mean) = {:,.2f}'
print(txt3.format(coeffvartnmean))

txt4 = '\nTrimmed by 10% Mean = €{:,.2f}'
print(txt4.format(stats.trim_mean(y_temp,0.10)))

txt5 = '\nInter Quartile Range = €{:,.2f}'
print(txt5.format(stats.iqr(y_temp)))

#fnumber = stats.f_oneway(y_temp, TMC_array)
#txt6 = 'ANOVA results = F number is {:.2f}, SE of F number is {:.2f}'
#print(txt6.format(fnumber))

TMC_array = np.asarray(df['TotalMissionCost'])
corr = stats.pearsonr(y_temp, TMC_array)
txt7 = '\nCorrelation results = {:,.2f}'
# print(txt7.format(corr))

Sum_of_errors = []
Sum_of_errors = (y_temp - TMC_array)
soe_mean_tmc = np.mean(Sum_of_errors)
txt8 = '\nAverage Error in Euros = €{:,.2f}'
print(txt8.format(soe_mean_tmc))

error_percent_of_mean = np.mean(Sum_of_errors) / np.mean(TMC_array)
txt9 = "\nAverage Error in % of TotalMissionCost mean = {:,.2f}%"
print(txt9.format(error_percent_of_mean))

soe_mean_output = np.mean(Sum_of_errors) / np.mean(y_temp)
txt10 = "\nAverage Error in % of Model Output mean = {:,.2f}%"
print(txt10.format(soe_mean_output))

#array_of_errors = np.array(Sum_of_errors)
#se_mean_euro = stats.tsem(array_of_errors)
#txt11 = "\nStandard Error in Euros = €{:,.2f}"
#print(txt11.format(se_mean_euro))

se_errors_percent_mean_error = (stats.tsem(Sum_of_errors) / np.mean(Sum_of_errors))
txt12 = "\nStandard Error in % of Avg Error = {:,.5f}%"
print(txt12.format(se_errors_percent_mean_error))


# print('*** VERSION TWO OUTPUT ***')

TMC_array = np.asarray(df['TotalMissionCost'])
# full precision no formatting added
#print("\n'TotalMissionCost' statistics:'")
#print('\nDescribe summary statistics listing = \n',stats.describe(TMC_array))
#print('Standard Deviation = ',stats.tstd(TMC_array))
#print('Standard Error of the Mean = ',stats.tsem(TMC_array))
#print('Coefficient of Variation (stdev / mean) = ',stats.variation(TMC_array))
#print('Trimmed by 10% Mean = ',stats.trim_mean(TMC_array,0.10))
#print('Inter Quartile Range = ',stats.iqr(TMC_array))
#print('ANOVA results = ',stats.f_oneway(y_temp, TMC_array))
#print('Correlation results = ',stats.pearsonr(TMC_array,y_temp))
#Sum_of_errors = (TMC_array - y_temp)
#print('Average Error = ', np.mean(Sum_of_errors))
#print("Average Error in % of TotalMissionCost mean = ", np.mean(abs(Sum_of_errors)) / np.mean(TMC_array),'%')
#print("Average Error in % of Model Output mean= ", np.mean(abs(Sum_of_errors)) / np.mean(y_temp),'%')
#print("Standard Error in Euros = ", stats.tsem(Sum_of_errors))
#print("Standard Error in % of Avg Error= ", stats.tsem(Sum_of_errors) / np.mean(Sum_of_errors),'%')

# Formatting added to 2 decimal places
print("\n**** 'TotalMissionCost' statistics:' ***")

sdev = stats.tstd(TMC_array)
txt1 = '\nStandard Deviation = €{:,.2f}'
print(txt1.format(sdev))

serror = stats.tsem(TMC_array)
txt2 = '\nStandard Error of the Mean = €{:,.2f}'
print(txt2.format(serror))

coeffvartnmean = stats.variation(TMC_array)
txt3 = '\nCoefficient of Variation (stdev / mean) = {:,.2f}'
print(txt3.format(coeffvartnmean))

txt4 = '\nTrimmed by 10% Mean = €{:,.2f}'
print(txt4.format(stats.trim_mean(TMC_array,0.10)))

txt5 = '\nInter Quartile Range = €{:,.2f}'
print(txt5.format(stats.iqr(TMC_array)))

#fnumber = stats.f_oneway(y_temp, TMC_array)
#txt6 = 'ANOVA results = F number is {:.2f}, SE of F number is {:.2f}'
#print(txt6.format(fnumber))

corr = stats.pearsonr(TMC_array,y_temp)
txt7 = '\nCorrelation results = {:,.2f}'
# print(txt7.format(corr))

Sum_of_errors = []
Sum_of_errors = (TMC_array - y_temp)
soe_mean_tmc = np.mean(Sum_of_errors)
txt8 = '\nAverage Error in Euros = €{:,.2f}'
print(txt8.format(soe_mean_tmc))

soe_mean_output = np.mean(abs(Sum_of_errors)) / np.mean(y_temp)
txt10 = "\nAverage Error in % of Model Output mean = {:,.2f}%"
print(txt10.format(soe_mean_output))

error_percent_of_mean = np.mean(abs(Sum_of_errors)) / np.mean(TMC_array)
txt9 = "\nAverage Error in % of TotalMissionCost mean = {:,.2f}%"
print(txt9.format(error_percent_of_mean))

# array_of_errors = np.array(Sum_of_errors)
# se_mean_euro = stats.tsem(array_of_errors)
# txt11 = "\nStandard Error in Euros = €{:,.2f}"
# print(txt11.format(se_mean_euro))

se_errors_percent_mean_error = (stats.tsem(Sum_of_errors) / np.mean(Sum_of_errors))
txt12 = "\nStandard Error in % of Avg Error = {:,.5f}%"
print(txt12.format(se_errors_percent_mean_error))

res = stats.probplot(Sum_of_errors, plot=plt)
plt.figzise=(15,15)
plt.show()

plt.plot(Sum_of_errors)
plt.figzise=(15,15)
plt.show()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


IsADirectoryError: ignored

In [5]:
import os
print (os.path.abspath(os.curdir))


/content
